In [17]:
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import json
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import torch.backends.cudnn as cudnn
import random
from torch.utils.data import Dataset, DataLoader, TensorDataset
from scipy import stats
from sklearn.preprocessing import StandardScaler
from hynix.model_class import LSTM, LSTM_model
from pycaret.regression import *
from sklearn.utils import resample
import warnings
warnings.filterwarnings(action='ignore')

In [18]:
train = pd.read_csv('trainset.csv',index_col=0)
train

,ID,x1,x3,x4,x5,x6,x7,x8,x9,x10,...,x1479,x1480,x1481,x1482,x1483,x1484,x1485,x1486,x1548,Y
1,3932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.702280,87.71842
2,6640,NaN,23.3375,NaN,NaN,NaN,0.018519,NaN,NaN,854.589577,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,94.904213,86.37922
3,3941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.975949,87.91689
4,5234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.888067,87.07533
5,5506,NaN,NaN,NaN,NaN,NaN,-0.163333,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.345613,89.16883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,6092,NaN,23.4625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.020648,87.35844
2493,4385,NaN,NaN,NaN,NaN,202.160513,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.337581,87.11169
2494,3662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.857049,88.66493
2495,6220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.248762,89.56981


In [19]:
pd.read_csv("input_csvs/prepro_cgw.csv")

,ID,x9,x59,x98,x123,x124,x176,x205,x206,x207,...,x1452,x1453,x1457,x1458,x1460,x194,x195,x196,x1548,Y
0,2128,1.740000,25.218519,0.619028,25.147083,33.708866,23.568579,40.266667,3.25,30.233333,...,6.83,246.116667,22.833333,2.72,5.00,75.033333,0.32,94.983333,86.846030,89.07114
1,2318,1.194269,25.369209,-0.203690,26.379330,33.955633,21.582357,51.583333,9.37,31.233333,...,4.90,126.233333,21.166667,2.20,4.82,381.196638,0.33,479.382941,77.991250,88.42078
2,983,2.541473,24.257770,0.128521,23.429916,34.096420,22.989821,47.350000,1.55,36.166667,...,5.15,155.100000,21.433333,1.65,4.70,9.450000,0.50,39.466667,77.815150,89.11948
3,4256,2.056909,23.894265,-0.420163,26.383720,35.926128,23.048133,41.116667,5.92,119.526742,...,6.72,239.750000,36.416667,4.02,4.67,381.196638,0.30,53.633333,84.200625,88.23637
4,2756,0.803218,26.845161,0.613835,25.540618,35.383359,21.296462,184.866667,5.90,37.633333,...,4.45,114.400000,22.250000,1.67,4.58,40.850000,0.43,66.216667,83.661019,89.53247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,6349,0.855800,25.658076,1.679347,26.200569,34.680417,22.205951,136.650000,2.58,56.516667,...,5.75,205.360404,25.733333,4.40,4.08,31.000000,0.65,69.566667,76.089387,89.82338
1493,5839,0.742439,24.888791,-0.648576,26.494461,34.742929,22.491013,75.250000,3.12,47.516667,...,5.35,146.750000,36.566667,3.32,3.97,23.083333,0.38,45.650000,77.235289,90.12727
1494,3566,0.690563,24.231748,0.594243,26.128810,33.734739,21.093873,553.966667,2.63,529.550000,...,6.78,243.033333,35.766667,1.68,4.27,38.883333,0.47,67.050000,89.259687,87.95325
1495,5939,0.176169,25.645219,0.998273,26.327912,35.072778,22.542844,111.966667,6.23,87.650000,...,7.98,310.316667,22.266667,1.55,4.37,46.033333,0.38,68.916667,94.706366,86.41818


In [20]:
cols = pickle.load(open('models\gw/train_cols.pkl','rb'))

In [23]:
cols[188:1454]

['x205',
 'x206',
 'x207',
 'x208',
 'x209',
 'x210',
 'x211',
 'x200',
 'x201',
 'x202',
 'x203',
 'x295',
 'x296',
 'x530',
 'x531',
 'x532',
 'x533',
 'x534',
 'x535',
 'x528',
 'x529',
 'x538',
 'x539',
 'x540',
 'x541',
 'x536',
 'x537',
 'x542',
 'x543',
 'x544',
 'x545',
 'x546',
 'x547',
 'x524',
 'x525',
 'x526',
 'x527',
 'x198',
 'x199',
 'x216',
 'x217',
 'x218',
 'x219',
 'x220',
 'x221',
 'x222',
 'x223',
 'x224',
 'x225',
 'x236',
 'x237',
 'x246',
 'x247',
 'x238',
 'x239',
 'x234',
 'x235',
 'x240',
 'x241',
 'x242',
 'x243',
 'x244',
 'x245',
 'x289',
 'x290',
 'x291',
 'x292',
 'x293',
 'x294',
 'x212',
 'x213',
 'x214',
 'x215',
 'x262',
 'x263',
 'x270',
 'x271',
 'x264',
 'x265',
 'x226',
 'x227',
 'x266',
 'x267',
 'x268',
 'x269',
 'x325',
 'x326',
 'x327',
 'x328',
 'x329',
 'x228',
 'x229',
 'x232',
 'x233',
 'x250',
 'x251',
 'x248',
 'x249',
 'x256',
 'x257',
 'x230',
 'x231',
 'x252',
 'x253',
 'x254',
 'x255',
 'x258',
 'x259',
 'x260',
 'x261',
 'x282',
 